![dphi banner](https://dphi-courses.s3.ap-south-1.amazonaws.com/Datathons/dphi_banner.png)

# **Getting Started Code For [Pneumonia Classification in CXRs](https://dphi.tech/challenges/pneumonia-classification-in-cxrs/76/overview/about) on DPhi**

## Download the images
We are given google drive link in the '[Data](https://dphi.tech/challenges/pneumonia-classification-in-cxrs/76/data)' section of problem page which has all the required train images (to build the model) and test images to predict the label of these images and submit the predictions on the [DPhi platform](https://dphi.tech/challenges/pneumonia-classification-in-cxrs/76/submit).

We can use **GoogleDriveDownloader** form **google_drive_downloader** library in Python to download the shared files from the shared Google drive link: https://drive.google.com/file/d/1d_93d9oFNRBK9Vg6BRxs9wvRbKtNTylY/view?usp=sharing

The file id in the above link is: **1d_93d9oFNRBK9Vg6BRxs9wvRbKtNTylY**

In [2]:
%pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1d_93d9oFNRBK9Vg6BRxs9wvRbKtNTylY',
                                    dest_path='content/pneumonia_dataset.zip',
                                    unzip=True)

Note: you may need to restart the kernel to use updated packages.


We have all the files from the shared Google drive link downloaded in the colab environment.

## Loading Libraries
All Python capabilities are not loaded to our working environment by default (even they are already installed in your system). So, we import each and every library that we want to use.

We chose alias names for our libraries for the sake of our convenience (numpy --> np and pandas --> pd, tensorlow --> tf).

Note: You can import all the libraries that you think will be required or can import it as you go along.

In [3]:
!apt-get update
!apt-get install ffmpeg libsm6 libxext6 -y
%pip install pandas numpy opencv-python scikit-learn keras
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool
import os                                               # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set
from sklearn.metrics import accuracy_score
from keras.layers.normalization import BatchNormalization

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease                     
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]   
Fetched 252 kB in 1s (384 kB/s)                               
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.2-1).
libxext6 is already the newest version (2:1.3.3-1).
ffmp

## Loading and preparing training data
The train and test images are given in two different folders - 'train' and 'test'.

In [4]:
data=[]
img_size = 100
def create_data():
        for item in ['normal','pneumonia']:
            path='/home/content/pneumonia_dataset/train/' + item+"/"
            
            for img in os.listdir(path):         # os.listdir gets you all the list of name of files located in the given path
                try:
                    img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)    # converts the image to pixels and gray scales the images
                    new_img_array=cv2.resize(img_array,(img_size,img_size))
                    # print(img_array)
                    if item == 'normal':
                        data.append([new_img_array,0])
                    else:
                        data.append([new_img_array, 1]) # appending the list of image pixels and respective target value in data
                except Exception as e:
                    pass                                      # try and except is exception handling case in python, saves you from getting errors
                
            
create_data()

In [5]:
len(data)

2425

In [6]:
# image pixels of a image
data[2422]

[array([[ 0,  0,  0, ..., 11, 12, 13],
        [ 0,  0,  0, ..., 11, 12, 11],
        [ 0,  0,  0, ..., 10, 10, 10],
        ...,
        [ 0,  0,  6, ...,  5,  5,  5],
        [ 0,  0,  7, ...,  5,  5,  5],
        [ 0,  0,  6, ...,  5,  5,  5]], dtype=uint8),
 1]

#### Shuffle the data

In [7]:
np.random.shuffle(data)

#### Separating the images and labels


In [8]:
x = []
y = []
for image in data:
  x.append(image[0])
  y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

In [9]:
np.unique(y, return_counts=True)

(array([0, 1]), array([1280, 1145]))

#### Splitting the data into Train and Validation Set
We want to check the performance of the model that we built. For this purpose, we always split (both independent and dependent data) the given data into training set which will be used to train the model, and test set which will be used to check how accurately the model is predicting outcomes.

For this purpose we have a class called 'train_test_split' in the 'sklearn.model_selection' module.

In [10]:
## Convert into 4D Array
x =  x.reshape(-1, 100, 100, 1)

In [11]:
# split the data
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3, random_state = 42)

In [12]:
X_train.shape

(1697, 100, 100, 1)

In [13]:
X_train =  X_train.reshape(-1, 100, 100, 1)

In [14]:
X_train.shape

(1697, 100, 100, 1)

In [15]:
X_train[0]

array([[[188],
        [177],
        [168],
        ...,
        [166],
        [173],
        [181]],

       [[155],
        [ 77],
        [ 28],
        ...,
        [ 38],
        [ 41],
        [ 48]],

       [[ 23],
        [  0],
        [  0],
        ...,
        [  0],
        [  0],
        [  0]],

       ...,

       [[  9],
        [  7],
        [  6],
        ...,
        [  0],
        [  3],
        [  5]],

       [[  9],
        [  7],
        [  6],
        ...,
        [  0],
        [  3],
        [  5]],

       [[  9],
        [  7],
        [  6],
        ...,
        [  1],
        [  3],
        [  5]]], dtype=uint8)

## Building Model
Now we are finally ready, and we can train the model.

There are many machine learning or deep learning models like Random Forest, Decision Tree, Multi-Layer Perceptron (MLP), Convolution Neural Network (CNN), etc. to say you some.


Then we would feed the model both with the data (X_train) and the answers for that data (y_train)

In [16]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # tf.keras.layers.Flatten(input_shape=(100, 100, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [17]:
cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [18]:
cnn.fit(X_train, y_train, epochs=10)

Epoch 1/10
54/54 [==============================] - 30s 545ms/step - loss: 67.5186 - accuracy: 0.4939
Epoch 2/10
54/54 [==============================] - 29s 530ms/step - loss: 0.6333 - accuracy: 0.6562
Epoch 3/10
54/54 [==============================] - 29s 534ms/step - loss: 0.5671 - accuracy: 0.7164
Epoch 4/10
54/54 [==============================] - 30s 547ms/step - loss: 0.4447 - accuracy: 0.7805
Epoch 5/10
54/54 [==============================] - 29s 540ms/step - loss: 0.4110 - accuracy: 0.7976
Epoch 6/10
54/54 [==============================] - 29s 538ms/step - loss: 0.3396 - accuracy: 0.8479
Epoch 7/10
54/54 [==============================] - 30s 547ms/step - loss: 0.2157 - accuracy: 0.9146
Epoch 8/10
54/54 [==============================] - 29s 544ms/step - loss: 0.1673 - accuracy: 0.9356
Epoch 9/10
54/54 [==============================] - 29s 538ms/step - loss: 0.1231 - accuracy: 0.9587
Epoch 10/10
54/54 [==============================] - 29s 532ms/step - loss: 0.2578 - accur

## Validate the model
Wonder🤔 how well your model learned! Lets check its performance on the X_val data.

In [19]:
cnn.evaluate(X_val, y_val)

23/23 [==============================] - 3s 141ms/step - loss: 1.4931 - accuracy: 0.5769


[1.493087887763977, 0.5769230723381042]

## Predict The Output For Testing Dataset 😅
We have trained our model, evaluated it and now finally we will predict the output/target for the testing data.

#### Load Test Set
Load the test data on which final submission is to be made.

In [21]:
# Loading the order of the image's name that has been provided
test_image_order = pd.read_csv("/home/content/pneumonia_dataset/test.csv")
test_image_order.head()

,filename
0,CXR_test_519.png
1,CXR_test_578.png
2,CXR_test_359.png
3,CXR_test_573.png
4,CXR_test_471.png


#### Getting images file path

In [22]:
file_paths = [[fname, '/home/content/pneumonia_dataset/test/' + fname] for fname in test_image_order['filename']]

#### Confirm if number of images in test folder is same as number of image names in 'Testing_set_face_mask.csv'

In [23]:
# Confirm if number of images is same as number of labels given
if len(test_image_order) == len(file_paths):
    print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths))
else:
    print('Number of image names does not match the number of filepaths')

Number of image names i.e.  606 matches the number of file paths i.e.  606


#### Converting the file_paths to dataframe

In [24]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_images.head()

,filename,filepaths
0,CXR_test_519.png,/home/content/pneumonia_dataset/test/CXR_test_...
1,CXR_test_578.png,/home/content/pneumonia_dataset/test/CXR_test_...
2,CXR_test_359.png,/home/content/pneumonia_dataset/test/CXR_test_...
3,CXR_test_573.png,/home/content/pneumonia_dataset/test/CXR_test_...
4,CXR_test_471.png,/home/content/pneumonia_dataset/test/CXR_test_...


## Data Pre-processing on test_data


In [25]:
test_pixel_data = []     # initialize an empty numpy array
for i in range(len(test_images)):
  
  img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale
  new_img_array=cv2.resize(img_array,(img_size,img_size))
  test_pixel_data.append(new_img_array)

In [26]:
test_pixel_data = np.asarray(test_pixel_data)

In [27]:
test_pixel_data =  test_pixel_data.reshape(-1, 100, 100, 1)

In [28]:
test_pixel_data

array([[[[  3],
         [  3],
         [  3],
         ...,
         [  5],
         [  6],
         [  7]],

        [[  3],
         [  2],
         [  2],
         ...,
         [  5],
         [  6],
         [  6]],

        [[  4],
         [  2],
         [  2],
         ...,
         [  5],
         [  5],
         [  6]],

        ...,

        [[  7],
         [  5],
         [  3],
         ...,
         [100],
         [ 85],
         [ 64]],

        [[  7],
         [  4],
         [  3],
         ...,
         [106],
         [ 97],
         [ 71]],

        [[  5],
         [  3],
         [  3],
         ...,
         [103],
         [ 94],
         [ 72]]],


       [[[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [

### Make Prediction on Test Dataset
Time to make a submission!!!

In [29]:
pred = cnn.predict(test_pixel_data)

In [30]:
pred

array([[3.17954004e-01],
       [7.97620058e-01],
       [1.37961090e-01],
       [5.28553426e-01],
       [8.01626801e-01],
       [9.33149695e-01],
       [2.61726141e-01],
       [3.93977046e-01],
       [5.15856028e-01],
       [5.67422628e-01],
       [6.50611758e-01],
       [9.05259013e-01],
       [1.19031221e-01],
       [5.61412573e-02],
       [4.51773405e-01],
       [4.83163536e-01],
       [1.08336657e-01],
       [3.99291277e-01],
       [9.74836230e-01],
       [5.60470223e-02],
       [4.17891145e-01],
       [9.62538838e-01],
       [4.53553736e-01],
       [9.92555737e-01],
       [9.99774933e-01],
       [9.83669758e-01],
       [9.94538188e-01],
       [6.49280071e-01],
       [8.25261712e-01],
       [4.11980599e-01],
       [5.04623294e-01],
       [9.98991370e-01],
       [2.37740755e-01],
       [3.06748748e-02],
       [1.72639787e-02],
       [3.41032594e-01],
       [5.56193590e-01],
       [8.99727583e-01],
       [9.51996207e-01],
       [3.85341913e-01],


Convert the numerical classes to corresponding categorical classes.

In [31]:
predictions = []
for item in pred:
  if item <= 0.5:
    predictions.append('normal')
  else:
    predictions.append('pneumonia')

In [32]:
predictions

['normal',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'normal',
 'normal',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'pneumonia',
 'normal',
 'normal',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 'normal',
 'normal',
 'normal',
 'normal',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'normal',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'normal',
 'pneumonia',
 'normal',
 'normal',
 'normal',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'normal',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 'normal',
 'normal',
 'normal',
 'pneumonia',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 'normal',
 'normal',
 'normal',
 'pneumonia',
 'normal',
 'normal',
 'normal',
 'pneumo

## **How to save prediciton results locally via jupyter notebook?**
If you are working on Jupyter notebook, execute below block of codes. A file named 'submission.csv' will be created in your current working directory.

In [33]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)      # the csv file will be saved locally on the same location where this notebook is located.

# **OR,**
**If you are working on Google Colab then use the below set of code to save prediction results locally**

## **How to save prediction results locally via colab notebook?**
If you are working on Google Colab Notebook, execute below block of codes. A file named 'prediction_results' will be downloaded in your system.

In [ ]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False) 

# To download the csv file locally
from google.colab import files        
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Well Done! 👍**
You are all set to make a submission. Let's head to the **[challenge page](https://dphi.tech/challenges/pneumonia-classification-challenge-by-segmind/76/overview/about)** to make the submission.